In [25]:
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$


Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавай свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. 
Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками

In [5]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [31]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        self.categorical = categorical
        self.numeric = numeric


    def fit(self, X, y):
        X_fit = X.copy()
        y_fit = y.copy()
        self.target = y.name
        self.numeric = [x for x in X_fit.columns if x not in self.categorical]
        X_with_target = pd.concat((X_fit, y_fit), axis=1)

        self.dict_of_means = {col : (X_with_target.groupby(col)[self.target].mean() + random.uniform(0, 1) * 0.006) for col in self.categorical}

        return self

    def transform(self, df):

        temp = df.copy()
        for col in self.categorical:
            temp[col] = temp[col].map(self.dict_of_means[col])
            temp[col] = temp[col].fillna(0)

        return temp

In [35]:
### Проверка работы трансформера

np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
3294,13.474648,13.443615,50000,13.089913,12.622742,13.774011,12.974392
2290,12.120983,11.914208,70000,12.454138,12.622742,13.774011,12.974392
874,12.307291,13.340781,50000,12.454138,12.622742,12.640949,12.974392
1907,12.489625,13.061400,92198,12.454138,13.151965,12.640949,12.461646
3244,12.397480,12.864808,3240,12.454138,12.622742,12.640949,12.461646


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [29]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [37]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

for i in max_depth_list:
    for j in min_samples_split_list:
        for k in min_impurity_decrease_list:
            for l in max_leaf_nodes_list:
                pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ("decision_tree",
                  DecisionTreeRegressor(max_depth=i, min_samples_split=j, min_impurity_decrease=k, max_leaf_nodes = l))])

                pipe.fit(X_train, y_train)

                print(f'MSLE при {i}, {j}, {k}, {l} равно: {mse(y_test, pipe.predict(X_test)):.3f}' )



MSLE при 3, 10, 0, 100 равно: 0.797
MSLE при 3, 10, 0, 200 равно: 0.797
MSLE при 3, 10, 0, 500 равно: 0.797
MSLE при 3, 10, 0.1, 100 равно: 0.520
MSLE при 3, 10, 0.1, 200 равно: 0.520
MSLE при 3, 10, 0.1, 500 равно: 0.520
MSLE при 3, 10, 0.15, 100 равно: 0.520
MSLE при 3, 10, 0.15, 200 равно: 0.520
MSLE при 3, 10, 0.15, 500 равно: 0.520
MSLE при 3, 10, 0.2, 100 равно: 0.520
MSLE при 3, 10, 0.2, 200 равно: 0.520
MSLE при 3, 10, 0.2, 500 равно: 0.520
MSLE при 3, 50, 0, 100 равно: 0.797
MSLE при 3, 50, 0, 200 равно: 0.797
MSLE при 3, 50, 0, 500 равно: 0.797
MSLE при 3, 50, 0.1, 100 равно: 0.520
MSLE при 3, 50, 0.1, 200 равно: 0.520
MSLE при 3, 50, 0.1, 500 равно: 0.520
MSLE при 3, 50, 0.15, 100 равно: 0.520
MSLE при 3, 50, 0.15, 200 равно: 0.520
MSLE при 3, 50, 0.15, 500 равно: 0.520
MSLE при 3, 50, 0.2, 100 равно: 0.520
MSLE при 3, 50, 0.2, 200 равно: 0.520
MSLE при 3, 50, 0.2, 500 равно: 0.520
MSLE при 3, 100, 0, 100 равно: 0.797
MSLE при 3, 100, 0, 200 равно: 0.797
MSLE при 3, 100, 0, 

Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

splitter = TimeSeriesSplit(n_splits=3)

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ("decision_tree",
                  DecisionTreeRegressor())])


search = GridSearchCV(pipe,
                      param_grid,
                      scoring='neg_mean_squared_error')

search.fit(X_train, y_train)

print(f"Best parameter (CV score={search.best_score_:.2f}):")
print(search.best_params_)

print(f"Качество лучшей модели на финальном тесте: {search.score(X_test, y_test)}")



Best parameter (CV score=-0.57):
{'decision_tree__max_depth': 3, 'decision_tree__max_leaf_nodes': 500, 'decision_tree__min_impurity_decrease': 0.15, 'decision_tree__min_samples_split': 500}
Качество лучшей модели на финальном тесте: -0.5204871413036717


In [43]:
from sklearn.metrics import get_scorer_names

In [44]:
get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_